In [1]:
%load_ext autoreload 
%autoreload 2

import os
import gpflow
import numpy as np
import scipy.io
import tensorflow as tf

import matplotlib.pyplot as plt
import matplotlib as mpl
plt.style.use('ggplot')
mpl.rcParams['pgf.rcfonts'] = False

os.environ["CUDA_VISIBLE_DEVICES"] = ''

# this doesn't work for some reason...
# remove the default one just in case
# gpflow.get_session().close()

# gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.25)
# config = config=tf.ConfigProto(gpu_options=gpu_options)
# config.gpu_options.allow_growth=True
# sess = gpflow.session_manager.tf.Session(config=config)
# sess.as_default()
# print(sess)

from gp_svgp import GP_MNIST_SVGP
from mnist_cnn import CNN_MNIST
from mnist_svm import SVM_MNIST
from hybrid_mnist import Hybrid_MNIST
from mnist_preprocessing import get_mnist_data, preprocess_images

#from plotting import Plotter

Using TensorFlow backend.


In [2]:
class Data(object):
    def __init__(self):
        self.data = {}
    def write(self, path, value):
        ptr = self.data
        for key in path[:-1]:
            if key not in ptr:
                ptr[key] = {}
            ptr = ptr[key]
        ptr[path[-1]] = value
    def dump(self, name, path="../data/evaluation/"):
        np.save(path + name, self.data)
        print("Saved to file, emptying data container")
        self.data = {}

data = Data()

In [3]:
# Standard MNIST test data
((_, mnist_test_imgs), (ys_train_onehot, ys_test_onehot)) = get_mnist_data()
mnist_train_features = np.genfromtxt("../data/mnist_train_features.csv", delimiter=",")
mnist_test_features = np.genfromtxt("../data/mnist_test_features.csv", delimiter=",")

ys_train = np.argmax(ys_train_onehot, axis=1)
ys_test = np.argmax(ys_test_onehot, axis=1)
# easier to remember handle
correct_classes = ys_test
n = len(ys_test)

In [4]:
cnn_model = CNN_MNIST(isolate=True) # isolate sessions # get default pretrained, saved CNN

In [5]:
svm_model = SVM_MNIST(mnist_train_features, ys_train, nb_classes=10, retrain=False)

Loading SVM from ../models/svm/svm_SVM classifier_model.pkl


In [6]:
hybrid_models = []
for tol in np.arange(0, 2.0, 0.5):
    hybrid_models.append(Hybrid_MNIST(accept_cnn_tolerance = tol))
for tol in np.arange(0, 2.0, 0.5):
    hybrid_models.append(Hybrid_MNIST(accept_cnn_tolerance = tol, stronger_criterion=True))

In [7]:
# instantiate models we want to test
# only one at a time here to avoid interference

get_gp_model = [lambda: GP_MNIST_SVGP(kernel=gpflow.kernels.Linear, 
                         whitevar=0.1,
                         latent_split=25,
                         whiten=True,
                         q_diag=True,
                         minibatch=8000,
                         whitevar_trainable=False,
                         feat_trainable=True,
                         name="Linear Kernel GP", 
                         retrain=False),
                lambda: GP_MNIST_SVGP(kernel=gpflow.kernels.Matern12, 
                         whitevar=0.1,
                         latent_split=25,
                         whiten=True,
                         q_diag=True,
                         minibatch=8000,
                         whitevar_trainable=False,
                         feat_trainable=True,
                         name="Matern12 Kernel GP",
                         retrain=False),
                lambda: GP_MNIST_SVGP(kernel=gpflow.kernels.Matern32, 
                         whitevar=0.1,
                         latent_split=25,
                         whiten=True,
                         q_diag=True,
                         minibatch=8000,
                         whitevar_trainable=False,
                         feat_trainable=True,
                         name="Matern32 Kernel GP",
                         retrain=False),
                lambda: GP_MNIST_SVGP(kernel=gpflow.kernels.Matern52, 
                         whitevar=0.1,
                         latent_split=25,
                         whiten=True,
                         q_diag=True,
                         minibatch=8000,
                         whitevar_trainable=False,
                         feat_trainable=True,
                         name="Matern52 Kernel GP",
                         retrain=False),
                lambda: GP_MNIST_SVGP(kernel=gpflow.kernels.RBF, 
                         whitevar=0.1,
                         latent_split=25,
                         whiten=True,
                         q_diag=True,
                         minibatch=8000,
                         whitevar_trainable=False,
                         feat_trainable=True,
                         name="RBF Kernel GP",
                         retrain=False)
               ]


In [8]:
record_in = 'Standard MNIST'

# prediction accuracies
subkey = 'accuracy'


#CNN
mnist_cnn_preds = cnn_model.predict_img_batch(mnist_test_imgs)
mnist_cnn_classes = np.argmax(mnist_cnn_preds, axis=1)
mnist_cnn_correct = correct_classes == mnist_cnn_classes
mnist_cnn_accuracy = np.count_nonzero(mnist_cnn_correct) / n
print("CNN accuracy: ", mnist_cnn_accuracy)
data.write([record_in, subkey, cnn_model.name],mnist_cnn_accuracy)

# SVM
mnist_svm_classes = svm_model.predict_batch(mnist_test_features)
mnist_svm_correct = correct_classes == mnist_svm_classes
mnist_svm_accuracy = np.count_nonzero(mnist_svm_correct) / n
print("SVM accuracy: ", mnist_svm_accuracy)
data.write([record_in, subkey, svm_model.name], mnist_svm_accuracy)


CNN accuracy:  0.9911
SVM accuracy:  0.9937


In [11]:
# GP
for gp_model_getter in get_gp_model:
    gp_model = gp_model_getter()
    print("Instantiated", gp_model.name)
    mnist_gp_mu, mnist_gp_var = gp_model.predict_batch(mnist_test_features)
    mnist_gp_classes = np.argmax(mnist_gp_mu, axis=1)
    mnist_gp_correct = correct_classes == mnist_gp_classes
    mnist_gp_accuracy = np.count_nonzero(mnist_gp_correct) / n
    print(gp_model.name, "accuracy: ", mnist_gp_accuracy)
    data.write([record_in, subkey, gp_model.name], mnist_gp_accuracy)

    
    # analyze the overlap in mis-classifications
    mnist_cnn_incorrect = correct_classes != mnist_cnn_classes
    mnist_gp_incorrect = correct_classes != mnist_gp_classes
    mnist_overlap_incorrect = mnist_cnn_incorrect & mnist_gp_incorrect
    both_incorrect = np.count_nonzero(mnist_overlap_incorrect)
    print("Number incorrect in both models:", both_incorrect)
    print("Number CNN incorrect but not GP:", np.count_nonzero(mnist_cnn_incorrect) -  both_incorrect)
    print("Number GP incorrect but not CNN:", np.count_nonzero(mnist_gp_incorrect) -  both_incorrect)


    for hybrid in hybrid_models:
        name = hybrid.name + "(GP: {})".format(gp_model.name)
        hybrid_pred, hybrid_mus, hybrid_vars = hybrid.combine_predictions(mnist_cnn_preds, mnist_gp_mu, mnist_gp_var)
        hybrid_classes = hybrid_pred[:, 1]
        hybrid_correct = correct_classes == hybrid_classes
        hybrid_accuracy = np.count_nonzero(hybrid_correct) / n
        print(name, "accuracy:", hybrid_accuracy)
        data.write([record_in, subkey, name], hybrid_accuracy)
    del gp_model


Initialized Linear Kernel GP
Loading GP model from file ../models/gp/Linear Kernel GP_model.npy
Instantiated Linear Kernel GP
Linear Kernel GP accuracy:  0.9899
Number incorrect in both models: 74
Number CNN incorrect but not GP: 15
Number GP incorrect but not CNN: 27
Hybrid model tol=0.0 stronger_crit=False(GP: Linear Kernel GP) accuracy: 0.9899
Hybrid model tol=0.5 stronger_crit=False(GP: Linear Kernel GP) accuracy: 0.9896
Hybrid model tol=1.0 stronger_crit=False(GP: Linear Kernel GP) accuracy: 0.9902
Hybrid model tol=1.5 stronger_crit=False(GP: Linear Kernel GP) accuracy: 0.9908
Hybrid model tol=0.0 stronger_crit=True(GP: Linear Kernel GP) accuracy: 0.9899
Hybrid model tol=0.5 stronger_crit=True(GP: Linear Kernel GP) accuracy: 0.9896
Hybrid model tol=1.0 stronger_crit=True(GP: Linear Kernel GP) accuracy: 0.9902
Hybrid model tol=1.5 stronger_crit=True(GP: Linear Kernel GP) accuracy: 0.9908
Initialized Matern12 Kernel GP
Loading GP model from file ../models/gp/Matern12 Kernel GP_model

In [11]:
data.dump("_".join([record_in, subkey]))

Saved to file, emptying data container


In [12]:
# ---- White Noise + NMNIST ----

# Preprocess the N-MNIST data just like standard MNIST
nmnist_noisy = scipy.io.loadmat('../data/raw/n-mnist/nmnist-awgn.mat')

awgn_x_test = preprocess_images(nmnist_noisy['test_x'])
awgn_y_test_onehot = nmnist_noisy['test_y']
awgn_y_test = np.argmax(awgn_y_test_onehot, axis=1)
awgn_correct_classes = awgn_y_test

In [13]:
record_in = 'MNIST + AWGN'
subkey = 'Accuracy'

awgn_feats = cnn_model.extract_features(awgn_x_test)

#CNN
awgn_cnn_preds = cnn_model.predict_img_batch(awgn_x_test)
awgn_cnn_classes = np.argmax(awgn_cnn_preds, axis=1)
awgn_cnn_correct = awgn_correct_classes == awgn_cnn_classes
awgn_cnn_accuracy = np.count_nonzero(awgn_cnn_correct) / n
print("CNN accuracy: ", awgn_cnn_accuracy)
data.write([record_in, subkey, cnn_model.name], awgn_cnn_accuracy)

# SVM
awgn_svm_classes = svm_model.predict_batch(awgn_feats)
awgn_svm_correct = awgn_correct_classes == awgn_svm_classes
awgn_svm_accuracy = np.count_nonzero(awgn_svm_correct) / n
print("SVM accuracy: ", awgn_svm_accuracy)
data.write([record_in, subkey, svm_model.name], awgn_svm_accuracy)

CNN accuracy:  0.9538
SVM accuracy:  0.962


In [14]:
# GP
for gp_model_getter in get_gp_model:
    gp_model = gp_model_getter()
    print("Instantiated", gp_model.name)
    awgn_gp_mu, awgn_gp_var = gp_model.predict_batch(awgn_feats)
    awgn_gp_classes = np.argmax(awgn_gp_mu, axis=1)
    awgn_gp_correct = awgn_correct_classes == awgn_gp_classes
    awgn_gp_accuracy = np.count_nonzero(awgn_gp_correct) / n
    print(gp_model.name, "accuracy: ", awgn_gp_accuracy)
    data.write([record_in, subkey, gp_model.name], awgn_gp_accuracy)

    for hybrid in hybrid_models:
        name = hybrid.name + "(GP: {})".format(gp_model.name)
        hybrid_pred, hybrid_mus, hybrid_vars = hybrid.combine_predictions(awgn_cnn_preds, awgn_gp_mu, awgn_gp_var)
        hybrid_classes = hybrid_pred[:, 1]
        hybrid_correct = awgn_correct_classes == hybrid_classes
        hybrid_accuracy = np.count_nonzero(hybrid_correct) / n
        print(name, "accuracy:", hybrid_accuracy)
        data.write([record_in, subkey, name], hybrid_accuracy)
    del gp_model # Clear TF sessions

Initialized Linear Kernel GP
Loading GP model from file ../models/gp/Linear Kernel GP_model.npy
Instantiated Linear Kernel GP
Linear Kernel GP accuracy:  0.9488
Hybrid model tol=0.0 stronger_crit=False(GP: Linear Kernel GP) accuracy: 0.9487
Hybrid model tol=0.5 stronger_crit=False(GP: Linear Kernel GP) accuracy: 0.9512
Hybrid model tol=1.0 stronger_crit=False(GP: Linear Kernel GP) accuracy: 0.9527
Hybrid model tol=1.5 stronger_crit=False(GP: Linear Kernel GP) accuracy: 0.9534
Hybrid model tol=0.0 stronger_crit=True(GP: Linear Kernel GP) accuracy: 0.9488
Hybrid model tol=0.5 stronger_crit=True(GP: Linear Kernel GP) accuracy: 0.9509
Hybrid model tol=1.0 stronger_crit=True(GP: Linear Kernel GP) accuracy: 0.9527
Hybrid model tol=1.5 stronger_crit=True(GP: Linear Kernel GP) accuracy: 0.9534
Initialized Matern12 Kernel GP
Loading GP model from file ../models/gp/Matern12 Kernel GP_model.npy
Instantiated Matern12 Kernel GP
Matern12 Kernel GP accuracy:  0.964
Hybrid model tol=0.0 stronger_crit=

In [15]:
data.dump("_".join([record_in, subkey]))

Saved to file, emptying data container


In [16]:
nmnist_blur = scipy.io.loadmat('../data/raw/n-mnist/nmnist-blur.mat')
blur_x_test = preprocess_images(nmnist_blur['test_x'])
blur_y_test_onehot = nmnist_blur['test_y']
blur_y_test = np.argmax(blur_y_test_onehot, axis=1)
blur_correct_classes = blur_y_test

blur_feats = cnn_model.extract_features(blur_x_test)

In [17]:
record_in = 'MNIST + blur'
subkey = 'Accuracy'

blur_feats = cnn_model.extract_features(blur_x_test)

#CNN
blur_cnn_preds = cnn_model.predict_img_batch(blur_x_test)
blur_cnn_classes = np.argmax(blur_cnn_preds, axis=1)
blur_cnn_correct = blur_correct_classes == blur_cnn_classes
blur_cnn_accuracy = np.count_nonzero(blur_cnn_correct) / n
print("CNN accuracy: ", blur_cnn_accuracy)
data.write([record_in, subkey, cnn_model.name], blur_cnn_accuracy)

# SVM
blur_svm_classes = svm_model.predict_batch(blur_feats)
blur_svm_correct = blur_correct_classes == blur_svm_classes
blur_svm_accuracy = np.count_nonzero(blur_svm_correct) / n
print("SVM accuracy: ", blur_svm_accuracy)
data.write([record_in, subkey, svm_model.name], blur_svm_accuracy)

CNN accuracy:  0.9458
SVM accuracy:  0.9279


In [18]:
# GP
for gp_model_getter in get_gp_model:
    gp_model = gp_model_getter()
    print("Instantiated", gp_model.name)
    blur_gp_mu, blur_gp_var = gp_model.predict_batch(blur_feats)
    blur_gp_classes = np.argmax(blur_gp_mu, axis=1)
    blur_gp_correct = blur_correct_classes == blur_gp_classes
    blur_gp_accuracy = np.count_nonzero(blur_gp_correct) / n
    print(gp_model.name, "accuracy: ", blur_gp_accuracy)
    data.write([record_in, subkey, gp_model.name], blur_gp_accuracy)

    for hybrid in hybrid_models:
        name = hybrid.name + "(GP: {})".format(gp_model.name)
        hybrid_pred, hybrid_mus, hybrid_vars = hybrid.combine_predictions(blur_cnn_preds, blur_gp_mu, blur_gp_var)
        hybrid_classes = hybrid_pred[:, 1]
        hybrid_correct = blur_correct_classes == hybrid_classes
        hybrid_accuracy = np.count_nonzero(hybrid_correct) / n
        print(name, "accuracy:", hybrid_accuracy)
        data.write([record_in, subkey, name], hybrid_accuracy)
    del gp_model # Clear TF session, graph (hopefully)

Initialized Linear Kernel GP
Loading GP model from file ../models/gp/Linear Kernel GP_model.npy
Instantiated Linear Kernel GP
Linear Kernel GP accuracy:  0.9379
Hybrid model tol=0.0 stronger_crit=False(GP: Linear Kernel GP) accuracy: 0.9381
Hybrid model tol=0.5 stronger_crit=False(GP: Linear Kernel GP) accuracy: 0.9423
Hybrid model tol=1.0 stronger_crit=False(GP: Linear Kernel GP) accuracy: 0.9453
Hybrid model tol=1.5 stronger_crit=False(GP: Linear Kernel GP) accuracy: 0.946
Hybrid model tol=0.0 stronger_crit=True(GP: Linear Kernel GP) accuracy: 0.9379
Hybrid model tol=0.5 stronger_crit=True(GP: Linear Kernel GP) accuracy: 0.9417
Hybrid model tol=1.0 stronger_crit=True(GP: Linear Kernel GP) accuracy: 0.9454
Hybrid model tol=1.5 stronger_crit=True(GP: Linear Kernel GP) accuracy: 0.9461
Initialized Matern12 Kernel GP
Loading GP model from file ../models/gp/Matern12 Kernel GP_model.npy
Instantiated Matern12 Kernel GP
Matern12 Kernel GP accuracy:  0.8746
Hybrid model tol=0.0 stronger_crit=

In [19]:
data.dump('_'.join([record_in,  subkey]))

Saved to file, emptying data container


In [20]:
nmnist_noisy_lowcontrast = scipy.io.loadmat('../data/raw/n-mnist/nmnist-contrast.mat')
record_in = 'Low Contrast MNIST'

lowcontrast_x_test = preprocess_images(nmnist_noisy_lowcontrast['test_x'])
lowcontrast_y_test_onehot = nmnist_noisy_lowcontrast['test_y']
lowcontrast_y_test = np.argmax(lowcontrast_y_test_onehot, axis=1)
lowcontrast_correct_classes = lowcontrast_y_test

lowcontrast_feats = cnn_model.extract_features(lowcontrast_x_test)

In [21]:
record_in = 'MNIST + low contrast + AWGN'
subkey = 'Accuracy'

lowcontrast_feats = cnn_model.extract_features(lowcontrast_x_test)

#CNN
lowcontrast_cnn_preds = cnn_model.predict_img_batch(lowcontrast_x_test)
lowcontrast_cnn_classes = np.argmax(lowcontrast_cnn_preds, axis=1)
lowcontrast_cnn_correct = lowcontrast_correct_classes == lowcontrast_cnn_classes
lowcontrast_cnn_accuracy = np.count_nonzero(lowcontrast_cnn_correct) / n
print("CNN accuracy: ", lowcontrast_cnn_accuracy)
data.write([record_in, subkey, cnn_model.name], lowcontrast_cnn_accuracy)

# SVM
lowcontrast_svm_classes = svm_model.predict_batch(lowcontrast_feats)
lowcontrast_svm_correct = lowcontrast_correct_classes == lowcontrast_svm_classes
lowcontrast_svm_accuracy = np.count_nonzero(lowcontrast_svm_correct) / n
print("SVM accuracy: ", lowcontrast_svm_accuracy)
data.write([record_in, subkey, svm_model.name], lowcontrast_svm_accuracy)

CNN accuracy:  0.7715
SVM accuracy:  0.7003


In [22]:
# GP
for gp_model_getter in get_gp_model:
    gp_model = gp_model_getter()
    print("Instantiated", gp_model.name)
    lowcontrast_gp_mu, lowcontrast_gp_var = gp_model.predict_batch(lowcontrast_feats)
    lowcontrast_gp_classes = np.argmax(lowcontrast_gp_mu, axis=1)
    lowcontrast_gp_correct = lowcontrast_correct_classes == lowcontrast_gp_classes
    lowcontrast_gp_accuracy = np.count_nonzero(lowcontrast_gp_correct) / n
    print(gp_model.name, "accuracy: ", lowcontrast_gp_accuracy)
    data.write([record_in, subkey, gp_model.name], lowcontrast_gp_accuracy)

    for hybrid in hybrid_models:
        name = hybrid.name + "(GP: {})".format(gp_model.name)
        hybrid_pred, hybrid_mus, hybrid_vars = hybrid.combine_predictions(lowcontrast_cnn_preds, lowcontrast_gp_mu, lowcontrast_gp_var)
        hybrid_classes = hybrid_pred[:, 1]
        hybrid_correct = lowcontrast_correct_classes == hybrid_classes
        hybrid_accuracy = np.count_nonzero(hybrid_correct) / n
        print(name, "accuracy:", hybrid_accuracy)
        data.write([record_in, subkey, name], hybrid_accuracy)
    del gp_model # Clear TF session, graph (hopefully)

Initialized Linear Kernel GP
Loading GP model from file ../models/gp/Linear Kernel GP_model.npy
Instantiated Linear Kernel GP
Linear Kernel GP accuracy:  0.7779
Hybrid model tol=0.0 stronger_crit=False(GP: Linear Kernel GP) accuracy: 0.7707
Hybrid model tol=0.5 stronger_crit=False(GP: Linear Kernel GP) accuracy: 0.7706
Hybrid model tol=1.0 stronger_crit=False(GP: Linear Kernel GP) accuracy: 0.7714
Hybrid model tol=1.5 stronger_crit=False(GP: Linear Kernel GP) accuracy: 0.7714
Hybrid model tol=0.0 stronger_crit=True(GP: Linear Kernel GP) accuracy: 0.7779
Hybrid model tol=0.5 stronger_crit=True(GP: Linear Kernel GP) accuracy: 0.7775
Hybrid model tol=1.0 stronger_crit=True(GP: Linear Kernel GP) accuracy: 0.7742
Hybrid model tol=1.5 stronger_crit=True(GP: Linear Kernel GP) accuracy: 0.7724
Initialized Matern12 Kernel GP
Loading GP model from file ../models/gp/Matern12 Kernel GP_model.npy
Instantiated Matern12 Kernel GP
Matern12 Kernel GP accuracy:  0.7152
Hybrid model tol=0.0 stronger_crit

In [23]:
data.dump('_'.join([record_in, subkey]))

Saved to file, emptying data container


In [8]:
import gc
gc.collect()

1030

In [9]:
record_in = 'FGSM Adverserial MNIST'
#subkey = eps
subkey = 'Accuracy'
#----- Adverserial Examples -----

epsilons = np.arange(0.325, 0.4, 0.025)

for eps in epsilons:
    adv_cnn_predictions, perturbed = cnn_model.fgsm_attack(mnist_test_imgs, epsilon=eps)
    adv_cnn_classes = np.argmax(adv_cnn_predictions, axis=1)
    adv_fsgm_features = cnn_model.extract_features(perturbed)

    #print("Incorrect: ", np.count_nonzero(correct_classes != adv_cnn_classes))
    print("Epsilon =", eps)    
    
    #CNN performance
    adv_cnn_correct = adv_cnn_classes == correct_classes    
    adv_cnn_accuracy = np.count_nonzero(adv_cnn_correct) / n
    print(cnn_model.name, "accuracy: ", adv_cnn_accuracy)
    data.write([record_in, "epsilon={}".format(eps), cnn_model.name], adv_cnn_accuracy)
    
    # Evaluate SVM 
    adv_svm_preds = svm_model.predict_batch(adv_fsgm_features)
    # SVM performance
    adv_svm_correct = adv_svm_preds == correct_classes
    adv_svm_accuracy = np.count_nonzero(adv_svm_correct) / n
    print(svm_model.name, "accuracy: ", adv_svm_accuracy)
    data.write([record_in, "epsilon={}".format(eps), subkey, svm_model.name], adv_svm_accuracy)
    
    for gp_model_getter in get_gp_model:
        gp_model = gp_model_getter()
        print("Instantiated", gp_model.name)
        # Evaluate GP
        (adv_mu, adv_var) = gp_model.predict_batch(adv_fsgm_features)
        adv_gp_preds = np.argmax(adv_mu, axis=1)
        # GP performance
        adv_gp_correct = correct_classes == adv_gp_preds
        adv_gp_accuracy = np.count_nonzero(adv_gp_correct) / n
        print(gp_model.name, "accuracy: ", adv_gp_accuracy)
        data.write([record_in, "epsilon={}".format(eps), subkey, gp_model.name], adv_gp_accuracy)

#         lowcontrast_gp_mu, lowcontrast_gp_var = gp_model.predict_batch(lowcontrast_feats)
#         lowcontrast_gp_classes = np.argmax(lowcontrast_gp_mu, axis=1)
#         lowcontrast_gp_correct = lowcontrast_correct_classes == lowcontrast_gp_classes
#         lowcontrast_gp_accuracy = np.count_nonzero(lowcontrast_gp_correct) / n
#         print(gp_model.name, "accuracy: ", lowcontrast_gp_accuracy)
#         data.write([record_in, subkey, gp_model.name], lowcontrast_gp_accuracy)

        for hybrid in hybrid_models:
            name = hybrid.name + "(GP: {})".format(gp_model.name)
            hybrid_adv_pred, hybrid_adv_mus, hybrid_adv_vars = hybrid.combine_predictions(adv_cnn_predictions, adv_mu, adv_var)
            hybrid_adv_classes = hybrid_adv_pred[:, 1]
            hybrid_adv_correct = correct_classes == hybrid_adv_classes
            hybrid_adv_accuracy = np.count_nonzero(hybrid_adv_correct) / n
            print(name, "accuracy:", hybrid_adv_accuracy)
            data.write([record_in,  "epsilon={}".format(eps), subkey, name], hybrid_adv_accuracy)
        del gp_model # Clear TF session, graph (hopefully)
    data.dump('_'.join([record_in, "eps={}".format(eps), subkey]))
    gc.collect()

Epsilon = 0.325
CNN classifier accuracy:  0.172
SVM classifier accuracy:  0.1702
Initialized Linear Kernel GP
Loading GP model from file ../models/gp/Linear Kernel GP_model.npy
Instantiated Linear Kernel GP
Linear Kernel GP accuracy:  0.1656
Hybrid model tol=0.0 stronger_crit=False(GP: Linear Kernel GP) accuracy: 0.1668
Hybrid model tol=0.5 stronger_crit=False(GP: Linear Kernel GP) accuracy: 0.1691
Hybrid model tol=1.0 stronger_crit=False(GP: Linear Kernel GP) accuracy: 0.1687
Hybrid model tol=1.5 stronger_crit=False(GP: Linear Kernel GP) accuracy: 0.1703
Hybrid model tol=0.0 stronger_crit=True(GP: Linear Kernel GP) accuracy: 0.1656
Hybrid model tol=0.5 stronger_crit=True(GP: Linear Kernel GP) accuracy: 0.1678
Hybrid model tol=1.0 stronger_crit=True(GP: Linear Kernel GP) accuracy: 0.1689
Hybrid model tol=1.5 stronger_crit=True(GP: Linear Kernel GP) accuracy: 0.1701
Initialized Matern12 Kernel GP
Loading GP model from file ../models/gp/Matern12 Kernel GP_model.npy
Instantiated Matern12 

Epsilon = 0.375
CNN classifier accuracy:  0.1007
SVM classifier accuracy:  0.0994
Initialized Linear Kernel GP
Loading GP model from file ../models/gp/Linear Kernel GP_model.npy
Instantiated Linear Kernel GP
Linear Kernel GP accuracy:  0.0975
Hybrid model tol=0.0 stronger_crit=False(GP: Linear Kernel GP) accuracy: 0.0969
Hybrid model tol=0.5 stronger_crit=False(GP: Linear Kernel GP) accuracy: 0.0971
Hybrid model tol=1.0 stronger_crit=False(GP: Linear Kernel GP) accuracy: 0.0997
Hybrid model tol=1.5 stronger_crit=False(GP: Linear Kernel GP) accuracy: 0.0998
Hybrid model tol=0.0 stronger_crit=True(GP: Linear Kernel GP) accuracy: 0.0975
Hybrid model tol=0.5 stronger_crit=True(GP: Linear Kernel GP) accuracy: 0.0975
Hybrid model tol=1.0 stronger_crit=True(GP: Linear Kernel GP) accuracy: 0.1
Hybrid model tol=1.5 stronger_crit=True(GP: Linear Kernel GP) accuracy: 0.1
Initialized Matern12 Kernel GP
Loading GP model from file ../models/gp/Matern12 Kernel GP_model.npy
Instantiated Matern12 Kerne

Hybrid model tol=1.5 stronger_crit=True(GP: RBF Kernel GP) accuracy: 0.096
Saved to file, emptying data container
